In [ ]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

category = "lungs"
tradeoffs = [0.16237767391887217]

Error: Kernel is dead

In [14]:
def optimal_function(data_frame):
    return data_frame.shape[0] ** 2 * data_frame.shape[1]

In [19]:
def remove_nans(data_category, raw_nan_matrix, seqn_variable_trade_off):
    nan_matrix = raw_nan_matrix.copy()

    while nan_matrix.shape[0] * nan_matrix.shape[0] > 0 and nan_matrix.sum().sum() > 0:
        percentage_nan_per_variable = nan_matrix.sum() / nan_matrix.shape[0]
        percentage_nan_per_seqn = nan_matrix.T.sum() / nan_matrix.shape[1]

        max_percentage_nan_per_variable = percentage_nan_per_variable.max()
        max_percentage_nan_per_seqn = percentage_nan_per_seqn.max()

        if max_percentage_nan_per_seqn / max_percentage_nan_per_variable > seqn_variable_trade_off:
            nan_matrix.drop(index=nan_matrix.index[percentage_nan_per_seqn == max_percentage_nan_per_seqn], inplace=True)
        else:
            nan_matrix.drop(columns=nan_matrix.columns[percentage_nan_per_variable == max_percentage_nan_per_variable], inplace=True)

    if nan_matrix.shape[0] * nan_matrix.shape[0] > 0:
        cleaned_data_category = data_category.loc[nan_matrix.index, nan_matrix.columns]
        
        none_object_columns = cleaned_data_category.columns[cleaned_data_category.dtypes != "object"]
        return cleaned_data_category.drop(columns=none_object_columns[cleaned_data_category[none_object_columns].std() == 0])
    else:
        return pd.DataFrame()

In [20]:
data_category = pd.read_feather(f"../fusion/data/examination/{category}.feather").set_index("SEQN")
object_column =  data_category.columns[data_category.dtypes == "object"]
raw_nan_matrix = data_category.isna()
raw_nan_matrix[object_column] = (data_category[object_column] == str(np.nan)) | raw_nan_matrix[object_column]

results_on_trade_off = []

print("Initial shape:", data_category.shape)

for seqn_variable_trade_off in tqdm(tradeoffs):
    cleaned_data_category = remove_nans(data_category, raw_nan_matrix, seqn_variable_trade_off)        
    
    print(f"Shape of the cleaned data for a trade-off of {seqn_variable_trade_off}:", cleaned_data_category.shape)

    results_on_trade_off.append(optimal_function(cleaned_data_category))

print("Final scores:\n", results_on_trade_off)

if sum(results_on_trade_off) > 0:
    cleaned_data_category = remove_nans(data_category, raw_nan_matrix, seqn_variable_trade_offs[np.argmax(results_on_trade_off)]) 

    # cleaned_data_category.reset_index().to_feather(f"cleaning/data/examination/{category}.feather")
else:
    print("None of the preprocessing attempts worked...")       

100%|██████████| 1/1 [00:00<00:00,  7.35it/s]

Initial shape: (9252, 13)
Shape of the cleaned data for a trade-off of 0.16237767391887217: (448, 10)
Final scores:
 [2007040]


FileNotFoundError: [Errno 2] No such file or directory: 'cleaning/data/examination/liver.feather'